# 메소드와 데커레이터

## 간단한 데커레이터 예시

In [20]:
def check_is_admin(f):
    
    def wrapper(*args, **kwargs):
        if kwargs.get('username') != 'admin':
            raise Exception("This is not admin")
        return f(*args, **kwargs)
    return wrapper
    
class Store(object):
    
    @check_is_admin
    def set_food(self, username, food):
        #check_is_admin(username)
        return None

Store().set_food("user", "lemon")    

Exception: This is not admin

## 여러 데커레이터 사용하기
* 단일 함수를 사용하여 하나 이상의 데커레이터 사용하기

In [24]:
def check_user_is_not(username):
    def user_check_decorator(f):
        def wrapper(*args, **kwargs):
            if kwargs.get('username') != username:
                raise Exception(f'this is not allowed: {username}')
            return f(*args, **kwargs)
        return wrapper
    return user_check_decorator
    
class Store(object):

    @check_user_is_not("guest")   # 먼저 해석 
    @check_user_is_not("admin")
    def set_food(self, username, food):
        #check_is_admin(username)
        return None

Store().set_food("user", "lemon")   

Exception: this is not allowed: guest

## 클래스 데커레이터 

In [32]:
import uuid

def set_class_name_and_id(klass):
    klass.name = str(klass)
    klass.random_id = uuid.uuid4()
    return klass

@set_class_name_and_id
class SomeClass(object):
    pass

print(SomeClass.name, SomeClass.random_id)

<class '__main__.SomeClass'> efc10829-5056-42c5-b0e9-bcfa9912094d


## 함수를 클래스로 래핑하기 
* 상태 추적

In [40]:
class CountCalls(object):
    def __init__(self, f):
        self.f = f
        self.called = 0
    
    def __call__(self, *args, **kwargs):
        self.called += 1
        return self.f(*args, **kwargs)
    
@CountCalls
def print_hello():
    print('hello')

In [41]:
print_hello.called

0

In [42]:
print_hello()

hello


In [43]:
print_hello.called

1

## update_wrapper 로 원래 속성 검색

In [45]:
def foobar(username='someone'):
    """구현할 메소드 내용"""
    pass

print(foobar.__doc__, foobar.__name__)

구현할 메소드 내용 foobar


In [47]:
def check_is_admin(f):
    def wrapper(*args, **kwargs):
        # TODO
        return f(*args, **kwargs)
    return wrapper

@check_is_admin
def foobar(username='someone'):
    """구현할 메소드 내용"""
    pass

print(foobar.__doc__, foobar.__name__)

None wrapper


In [48]:
import functools

def check_is_admin(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        # TODO
        return f(*args, **kwargs)
    return wrapper

@check_is_admin
def foobar(username='someone'):
    """구현할 메소드 내용"""
    pass

print(foobar.__doc__, foobar.__name__)

구현할 메소드 내용 foobar


## inspect를 통해 kwarg 인수도 추출/사용

In [54]:
def test_func(a='1', b='2'):
    pass 

func_args = inspect.getcallargs(test_func)
func_args

{'a': '1', 'b': '2'}

In [55]:
import functools
import inspect

def check_is_admin(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        func_args = inspect.getcallargs(f, *args, **kwargs)
        print(func_args)
        if func_args.get('username') != 'admin':
            raise Exception("This is not admin")
        return f(*args, **kwargs)
    return wrapper

@check_is_admin
def foobar(username='someone'):
    """구현할 메소드 내용"""
    pass 

foobar('user')

{'username': 'user'}


Exception: This is not admin

## 파이선. bound method

In [56]:
class Pizza(object):
    def __init__(self, size):
        self.size = size
    def get_size(self):
        return self.size

In [57]:
Pizza.get_size   # unbounded

<function __main__.Pizza.get_size>

In [58]:
Pizza.get_size()   # cannot call unbounded function

TypeError: get_size() missing 1 required positional argument: 'self'

In [59]:
Pizza.get_size(Pizza(42))

42

In [60]:
Pizza(42).get_size   # bounded method

<bound method Pizza.get_size of <__main__.Pizza object at 0x7f09907eac88>>

In [61]:
Pizza(42).get_size()

42

## 정적 메소드

In [69]:
class Pizza(object):
    @staticmethod
    def mix_ingredients(x, y):
        return x + y
    
    def cook_with(self, x):
        pass

In [70]:
Pizza().cook_with is Pizza().cook_with

False

In [71]:
Pizza().mix_ingredients is Pizza().mix_ingredients

True

In [72]:
Pizza().mix_ingredients is Pizza.mix_ingredients

True

## 클래스 메서드

In [89]:
class Pizza(object):
    radius = 42  # 클래스에 포함
    
    @classmethod
    def get_radius(cls):
        return cls.radius

In [90]:
Pizza.get_radius

<bound method Pizza.get_radius of <class '__main__.Pizza'>>

In [91]:
Pizza().get_radius

<bound method Pizza.get_radius of <class '__main__.Pizza'>>

In [92]:
Pizza.get_radius is Pizza().get_radius

False

In [93]:
Pizza().get_radius()

<class '__main__.Pizza'>


42

## 클래스 메소드를 이용한 Factory 패턴

In [101]:
class Fridge(object):
    def __init__(self, cheese, vegetable):
        self.cheese = cheese
        self.vegetable = vegetable

class Pizza(object):
    
    def __init__(self, ingradients):
        self.ingradients = ingradients
        
    def cook(self):
        print(self.ingradients)
    
    @classmethod
    def from_fridge(cls, fridge):
        return cls(fridge.cheese + fridge.vegetable)
    
fridge = Fridge('cheese', 'vegetable')
Pizza.from_fridge(fridge).cook()

cheesevegetable


## 추상메소드 

In [103]:
import abc

class BasePizza(object, metaclass=abc.ABCMeta):
    
    @abc.abstractmethod
    def get_radius(self):
        pass
    
BasePizza()

TypeError: Can't instantiate abstract class BasePizza with abstract methods get_radius

In [109]:
class DietPizza(BasePizza):
    
    def get_radius(self):
        print('as a general method')
        pass

DietPizza().get_radius()

as a general method


In [110]:
class DietPizza(BasePizza):
    
    @staticmethod
    def get_radius():
        print('as a static method')
        pass

DietPizza().get_radius()

as a static method


In [111]:
class DietPizza(BasePizza):
    
    @classmethod
    def get_radius(cls):
        print('as a class method')
        pass

DietPizza().get_radius()

as a class method


## 다중상속시 super() 호출

In [114]:
class A(object):
    bar = 42
    def foo(self):
        pass
    
class B(object):
    bar = 0
    
class C(A,B):
    xyz = 'abc'

In [115]:
C.mro()  # method resolution order

[__main__.C, __main__.A, __main__.B, object]

In [116]:
super(C, C()).bar

42

In [117]:
super(C, C()).foo

<bound method A.foo of <__main__.C object at 0x7f09907b8e10>>

In [118]:
super(C)   # 두번째 인자가 bounding되지 않음 

<super: __main__.C, None>

In [119]:
super(C).bar # bounding이 필요 

AttributeError: 'super' object has no attribute 'bar'

In [120]:
super(C).foo # bounding이 필요 

AttributeError: 'super' object has no attribute 'foo'